In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

from pathlib import Path
import h5py

ModuleNotFoundError: No module named 'keras'

加载模型权重

In [ ]:
model_weight_resnet50 = ""
model_weight_xception = ""
model_weight_inceptionV3 = ""

建立辅助函数，提取特征向量

In [ ]:
def write_gap(MODEL, image_size, preprocessing=None, filename_option=""):
    # parameters
    batch_size = 64
    path_dataset = Path('dataset')
    path_train = path_dataset / "train2"
    path_test = path_dataset / "test2"
    
    # set up Model
    height = image_size[0]
    width = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    
    base_model = MODEL(input_tensor=x, weights=None, include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    # load weight
    model.load_weights("models/" + model_weight, by_name=True)
    
    train_gen = ImageDataGenerator(
        rotation_range=10.,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        zoom_range=0.05,
        preprocessing_function=preprocessing
    )
    
    test_gen = ImageDataGenerator(
        preprocessing_function=preprocessing
    )
    
    train_generator = train_gen.flow_from_directory(
        path_train, (height, weight), shuffle=False, batch_size=batch_size)
    test_generator = test_gen.flow_from_directory(
        path_test, (height, weight), shuffle=False, batch_size=batch_size)
    
    # predictions
    steps_train_sample = train_generator.samples // batch_size + 1
    steps_valid_sample = test_generator.samples // batch_size + 1
    train = model.predict_generator(train_generator, steps=steps_train_sample)
    test = model.predict_generator(test_generator, steps=steps_test_sample)
    
    # save features
    path_finetune_feature = 'features/finetune_{}{}.h5'.format(base_model.func_name, filename_option)
    with h5py.File(path_finetune_feature) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("valid", data=valid)
        h.create_dataset("label", data=train_generator.classes)
        h.create_dataset("valid_label", data=valid_generator.classes)
    print("write_gap {} finsh".format(Model.__name__))